In [1]:
## Libraries
import glob
from pathlib import Path

# - to extract gz files
import gzip
import json

# - to convert json to dataframe
import pandas as pd

In [2]:
## load config.ini
import configparser
config_file = '../config.ini'
settings = configparser.ConfigParser(inline_comment_prefixes="#")
settings.read(config_file)


['../config.ini']

In [7]:
path = settings['DEFAULT']['raw_data_folder'] + settings['LENS_API.PATENTS']['subfolder'] + '*'
files = glob.glob(path)

In [8]:
files

['../data/raw/patents\\au-patent-2020-01-01-2020-12-31.jsonl.gz',
 '../data/raw/patents\\au-patent-2021-01-01-2023-07-31.jsonl.gz',
 '../data/raw/patents\\au_patent_2020_01_01_to_2023_07_31.zip',
 '../data/raw/patents\\patents_2023-08-01_to_2023-08-31_0.json',
 '../data/raw/patents\\patents_2023-08-01_to_2023-08-31_from_0.json',
 '../data/raw/patents\\patents_2023-08-01_to_2023-08-31_from_100.json',
 '../data/raw/patents\\patents_2023-08-01_to_2023-08-31_from_200.json',
 '../data/raw/patents\\us-patent-2020-01-01-2020-01-27.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-01-28-2020-03-14.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-03-15-2020-04-30.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-05-01-2020-06-22.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-06-23-2020-08-07.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-08-08-2020-09-27.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-09-28-2020-11-14.jsonl.gz',
 '../data/raw/patents\\us-patent-2020-11-15-2020-12-31.jsonl.gz

In [2]:
patents_data = []
patents_classifications = []
patents_applicants = []
patents_inventors = []

In [3]:
file = '../data/raw/patents/patents_2023-08-01_to_2023-08-31_from_0.json'

In [14]:
f = open(file)


In [16]:
data = json.load(f)

In [ ]:
for d in data['data']:
    print(d['lens_id'])

In [29]:
file_reader = open(file)

In [32]:
f[0]

{'lens_id': '001-717-119-197-686',
 'date_published': '2023-08-01',
 'biblio': {'application_reference': {'doc_number': '202217652574'},
  'invention_title': [{'text': 'Method of manufacturing boltless unitary ring gear-flange body'}],
  'parties': {'applicants': [{'residence': 'US',
     'extracted_name': {'value': 'NPC ROBOTICS CORP'}}],
   'inventors': [{'residence': 'US',
     'sequence': 1,
     'extracted_name': {'value': 'REID RICHARD'}},
    {'residence': 'US',
     'sequence': 2,
     'extracted_name': {'value': 'DAVID JOHN'}},
    {'residence': 'US',
     'sequence': 3,
     'extracted_name': {'value': 'DICKIE PAUL'}}]},
  'classifications_cpc': {'classifications': [{'symbol': 'B23P15/14'},
    {'symbol': 'B21D22/16'},
    {'symbol': 'B21D53/28'},
    {'symbol': 'B23K20/12'},
    {'symbol': 'B23K2101/008'},
    {'symbol': 'F16H55/17'},
    {'symbol': 'F16H2055/176'},
    {'symbol': 'B21D53/28'},
    {'symbol': 'B21D22/14'},
    {'symbol': 'B23K2101/008'},
    {'symbol': 'B23P

In [31]:
patents_data = []
patents_classifications = []
patents_applicants = []
patents_inventors = []
file_ext = '.json'

if (file_ext =='.json'):
    file_reader = open(file)
    data = json.load(file_reader)
    f = data['data']
    file_reader.close()


for line in f:
    patent = ''
    if (file_ext =='.gz'):
        patent = json.loads(line)
    if (file_ext =='.json'):
        patent = line

    if patent['biblio'].get('invention_title') is None:         ## if invention title is None, then do not capture, skip to next record.
        continue                                                

    abstract = patent.get('abstract', 'na')
    if abstract != 'na':
        abstract = abstract[0]['text']
    data = {
        'lens_id': patent['lens_id'],
        'jurisdiction': patent['jurisdiction'],
        'patent_id': patent['doc_key'],
        'date_published': patent['date_published'],
        'title': patent['biblio']['invention_title'][0]['text'],
        'abstract': abstract
    }
    patents_data.append(data)

    for applicant in patent['biblio']['parties']['applicants']:
        app_data = {
            'lens_id': patent['lens_id'],
            'patent_id': patent['doc_key'],
            'residence': applicant.get('residence', 'NA'),
            'name': applicant['extracted_name']['value']
        }
        patents_applicants.append(app_data)
    
    if patent['biblio']['parties'].get('inventors') is not None:
        for inventor in patent['biblio']['parties']['inventors']:
            inv_data = {
                'lens_id': patent['lens_id'],
                'patent_id': patent['doc_key'],
                'residence': inventor.get('residence', 'NA'),
                'name': inventor['extracted_name']['value']
            }
            patents_inventors.append(inv_data)

    #print('process classifications')
    classifications_cpc = patent['biblio'].get('classifications_cpc')
    if classifications_cpc is not None:
        for classification in classifications_cpc['classifications']:
            #print('process classification')
            class_data = {
                'lens_id': patent['lens_id'],
                'patent_id': patent['doc_key'],
                'classification': classification['symbol']
            }
            patents_classifications.append(class_data)

    del(patent)    ## clear variable from memory
    del(line)
f.close()
del(f)

KeyError: 'jurisdiction'

In [23]:
test = ''
for line in f:
    test = line

NameError: name 'f' is not defined

In [35]:
def identify_new_files():
    ## read the raw files
    path = r'..\data\raw\patents\*.gz'
    files = glob.glob(path)
    s1 = pd.Series(files)

    ## read the processed log
    df = pd.read_csv('../data/meta/process_log/processed_patents.csv')
    s2 = df['processed files']

    ## compare and return pd Series

    s3 = s1[~ s1.isin(s2)]
    return s3

In [36]:
files = identify_new_files()

In [38]:
df = pd.read_csv('../data/meta/process_log/processed_patents.csv')

In [54]:
df2 = pd.concat([df['processed files'], files])

In [63]:
df3 = pd.DataFrame(pd.concat([df['processed files'], files]), columns=['processed files'])

In [64]:
df3

,processed files
0,..\data\raw\patents\au-patent-2020-01-01-2020-...
1,..\data\raw\patents\au-patent-2021-01-01-2023-...
2,..\data\raw\patents\us-patent-2020-01-01-2020-...
3,..\data\raw\patents\us-patent-2020-01-28-2020-...
4,..\data\raw\patents\us-patent-2020-03-15-2020-...
5,..\data\raw\patents\us-patent-2020-05-01-2020-...
6,..\data\raw\patents\us-patent-2020-06-23-2020-...
7,..\data\raw\patents\us-patent-2020-08-08-2020-...
8,..\data\raw\patents\us-patent-2020-09-28-2020-...
9,..\data\raw\patents\us-patent-2020-11-15-2020-...


In [1]:
def identify_new_files():
    ## read the raw files
    path = settings['DEFAULT']['raw_data_folder'] + settings['LENS_API.PATENTS']['subfolder'] + '*'
    files = glob.glob(path)
    s1 = pd.Series(files)

    ## read the processed log
    df = pd.read_csv('../data/meta/process_log/processed_patents.csv')
    s2 = df['processed files']

    ## compare and return pd Series

    s3 = s1[~ s1.isin(s2)]
    return s3

In [2]:
import glob
from pathlib import Path
import configparser
import argparse

# - to extract gz files
import gzip
import json

# - to convert json to dataframe
import pandas as pd

## load config.ini
config_file = '../config.ini'
settings = configparser.ConfigParser(inline_comment_prefixes="#")
settings.read(config_file)

['../config.ini']

In [3]:
files = identify_new_files()

In [4]:
file = files[0]

In [5]:
patents_data = []
patents_classifications = []
patents_applicants = []
patents_inventors = []
file_ext = Path(file).suffix


In [6]:
file_ext

'.json'

In [7]:
if (file_ext =='.json'):
    file_reader = open(file, encoding='utf-8')
    data = json.load(file_reader)
    f = data['data']
    file_reader.close()

JSONDecodeError: Expecting ',' delimiter: line 2 column 1 (char 75869)

In [11]:
file_reader = open(file, encoding="ascii", errors="ignore")

In [12]:
data = json.load(file_reader)

JSONDecodeError: Expecting ',' delimiter: line 2 column 1 (char 75861)

In [10]:
file_reader.close()

In [23]:
f = data['results']

In [24]:
f

{'docsMatched': 8269,
 'docs': [{'lensId': '004-925-244-801-702',
   'jurisdiction': 'US',
   'publicationDate': '2023-10-03',
   'filingNumber': 'US 201716463017 A',
   'title': 'Method for controlling or regulating a microscope illumination',
   'applicants': ['ZEISS CARL MICROSCOPY GMBH'],
   'inventors': ['GÖGLER MICHAEL', 'OHRT THOMAS'],
   'docType': 'Granted Patent',
   'cpcClassifications': ['G02B21/0096',
    'G02B21/06',
    'G02B21/365',
    'G02B21/0096',
    'G02B21/06',
    'G02B21/16',
    'G02B21/365'],
   'abstract': 'A method for regulating a light source of a microscope that illuminates an object, said method including specifying an intended value of an energy parameter of illumination radiation on the object; producing illumination radiation; providing an objective for focusing illumination radiation onto the object; ascertaining a transmission property of the objective for the illumination radiation; output coupling a component of the illumination radiation upstrea

In [6]:
import os
filename = '../data/raw/journals/journals.jsonl.gz'
os.system('gunzip ' + filename)


0

In [7]:
orders = ""
i = 0
e = 0 
f = open("../data/raw/journals/journals_2023-09-25_to_2023-09-29_from_0.json", "w")
try:
    with open('../data/raw/journals/journals_2023-09-25_to_2023-09-29_from_0_real.json', encoding="ascii", errors="ignore") as data:
        for each_line in data:
            try:
                if i == 0:
                    f.write('{"data":[')
                if i > 1000:
                    order = each_line.replace("\n", "]}").strip()
                    f.write(order)
                    f.close()
                    break
                order = each_line.replace("\n", ",").strip()
                #orders = orders + order
                f.write(order)
                i = i + 1
                

            except ValueError:
                e = e + 1
                pass

except:
    pass

f.close()

In [24]:
i

1001